In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 13-3. Flux.jl

※ 本文とは環境等を少し変えています（コードはほぼ同一です）

### コード13-12. `Flux.jl` 実験用の仮想環境（プロジェクト環境）構築

```julia
julia> mkdir("flux_wk") |> cd

julia> # `]` をタイプしてパッケージモードに移行

(@v1.7) pkg> activate .
  Activating new project at `/path/to/flux_wk`

(flux_wk) pkg> add Flux MLDatasets MLJBase ImageCore Statistics
    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General`
   Resolving package versions...
   # ：《以下略》、環境によっては時間がかかる場合があります

(flux_wk) pkg> st
      Status `/path/to/flux_wk/Project.toml`
  [587475ba] Flux v0.13.0
  [a09fc81d] ImageCore v0.9.3
  [eb30cadb] MLDatasets v0.5.16
  [a7f614a8] MLJBase v0.20.2
  [10745b16] Statistics
```

In [1]:
# mkdir("flux_wk") |> cd
mkpath("flux_wk") |> cd

In [2]:
]activate .

  Activating new project at `/path/to/JuliaBook2023Samples/notebooks/Chapter13/flux_wk`


In [3]:
]add Flux@0.13.0 MLDatasets@0.5.16 MLJBase@0.20.2 ImageCore@0.9.3 Statistics

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `git@github.com:JuliaRegistries/General.git`
   Resolving package versions...
   Installed ContextVariablesX ─ v0.1.3
   Installed ZipFile ─────────── v0.10.1
   Installed NNlibCUDA ───────── v0.2.6
   Installed Optimisers ──────── v0.2.9
   Installed Accessors ───────── v0.1.25
   Installed MLUtils ─────────── v0.2.11
   Installed NNlib ───────────── v0.8.16
   Installed DataFrames ──────── v1.3.6
   Installed StringEncodings ─── v0.3.6
   Installed WorkerUtilities ─── v1.6.1
   Installed MicroCollections ── v0.1.3
   Installed Graphics ────────── v1.1.2
   Installed BufferedStreams ─── v1.1.0
   Installed HDF5_jll ────────── v1.12.2+2
   Installed ChainRules ──────── v1.46.1
   Installed Zygote ──────────── v0.6.55
   Installed FLoops ──────────── v0.2.1
   Installed MosaicViews ─────── v0.3.4
   Installed BangBang ────────── v0.3.37
   Installed MLStyle ─────────── v0.4.16
   Installed JLD2 ────────────── v

  [76f85450] + LibGit2
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [d6f4376e] + Markdown
  [a63ad114] + Mmap
  [ca575930] + NetworkOptions v1.2.0
  [44cfe95a] + Pkg v1.8.0
  [de0858da] + Printf
  [3fa0cd96] + REPL
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [1a1011a3] + SharedArrays
  [6462fe0b] + Sockets
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
  [4607b0f0] + SuiteSparse
  [fa267f1f] + TOML v1.0.0
  [a4e569a6] + Tar v1.10.1
  [8dfed614] + Test
  [cf7118a7] + UUIDs
  [4ec0a83e] + Unicode
  [e66e0078] + CompilerSupportLibraries_jll v1.0.1+0
  [deac9b47] + LibCURL_jll v7.84.0+0
  [29816b5a] + LibSSH2_jll v1.10.2+0
  [c8ffd9c3] + MbedTLS_jll v2.28.0+0
  [14a3606d] + MozillaCACerts_jll v2022.2.1
  [4536629a] + OpenBLAS_jll v0.3.20+0
  [05823500] + OpenLibm_jll v0.8.1+0
  [83775a58] + Zlib_jll v1.2.12+3
  [8e850b90] + libblastrampoline_jll v5.1.1+0
  [8e850ede] + nghttp2_jll v1.48.0+0
  [3f19e933] + p7zip_jll v17

In [4]:
]st

Status `/path/to/JuliaBook2023Samples/notebooks/Chapter13/flux_wk/Project.toml`
⌃ [587475ba] Flux v0.13.0
⌃ [a09fc81d] ImageCore v0.9.3
⌃ [eb30cadb] MLDatasets v0.5.16
⌃ [a7f614a8] MLJBase v0.20.2
  [10745b16] Statistics
Info Packages marked with ⌃ have new versions available and may be upgradable.


### コード13-13. `Flux.jl` 使用例(1)：前準備

In [5]:
using Flux

using Flux: onehotbatch, onecold, throttle, @epochs

using Flux.Data: DataLoader

using MLDatasets.MNIST, Statistics

using MLJBase: accuracy

### コード13-14. `Flux.jl` 使用例(2)：ネットワーク（モデル）構築

In [6]:
model = Chain(
    Conv((5, 5), 1=>32, relu),
    MaxPool((2, 2)),
    Conv((5, 5), 32=>64, relu),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(1024, 1024, relu),
    Dropout(0.5),
    Dense(1024, 10),
    # softmax
) |> gpu

Unhandled Task ERROR: IOError: FDWatcher: bad file descriptor (EBADF)
Stacktrace:
 [1] try_yieldto(undo::typeof(Base.ensure_rescheduled))
   @ Base ./task.jl:871
 [2] wait()
   @ Base ./task.jl:931
 [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock})
   @ Base ./condition.jl:124
 [4] _wait(fdw::FileWatching._FDWatcher, mask::FileWatching.FDEvent)
   @ FileWatching ~/julia-1.8.5/share/julia/stdlib/v1.8/FileWatching/src/FileWatching.jl:535
 [5] wait(fdw::FileWatching.FDWatcher)
   @ FileWatching ~/julia-1.8.5/share/julia/stdlib/v1.8/FileWatching/src/FileWatching.jl:563
 [6] macro expansion
   @ ~/julia-1.8.5/share/julia/stdlib/v1.8/Downloads/src/Curl/Multi.jl:166 [inlined]
 [7] (::Downloads.Curl.var"#40#46"{Int32, FileWatching.FDWatcher, Downloads.Curl.Multi})()
   @ Downloads.Curl ./task.jl:484
┌ Warning: The NVIDIA driver on this system only supports up to CUDA 10.2.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA 

Chain(
  Conv((5, 5), 1 => 32, relu),          # 832 parameters
  MaxPool((2, 2)),
  Conv((5, 5), 32 => 64, relu),         # 51_264 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(1024 => 1024, relu),            # 1_049_600 parameters
  Dropout(0.5),
  Dense(1024 => 10),                    # 10_250 parameters
)                   # Total: 8 arrays, 1_111_946 parameters, 1.859 KiB.

### コード13-15. `Flux.jl` 使用例(3)：`loss` 関数の定義例

In [7]:
struct Loss{M} <: Function
    model::M
end

In [8]:
(loss::Loss)(x, y) = Flux.logitcrossentropy(loss.model(x), y)

In [9]:
(loss::Loss)((;data, label)) = loss(data, label)  # 要：Julia v1.7 以降

### コード13-16. `Flux.jl` 使用例(4)：MNISTデータセットのロードと `DataLoader` の使用例

In [10]:
function loadMNIST(::Type{T}, batchsize::Int = 1000) where {T<:AbstractFloat}
    train_x = reshape(MNIST.traintensor(T), 28, 28, 1, :) |> gpu
    train_y = onehotbatch(MNIST.trainlabels(), 0:9) |> gpu
    test_x = reshape(MNIST.testtensor(T), 28, 28, 1, :) |> gpu
    test_y = onehotbatch(MNIST.testlabels(), 0:9) |> gpu

    train_data = DataLoader((data=train_x, label=train_y), batchsize=batchsize, shuffle=true)
    (train_data, test_x, test_y)
end

loadMNIST (generic function with 2 methods)

In [11]:
batchsize = 1000;
traindata, tX, tY = loadMNIST(Float32, batchsize)

(DataLoader{NamedTuple{(:data, :label), Tuple{CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}, Flux.OneHotArray{UInt32, 10, 1, 2, CUDA.CuArray{UInt32, 1, CUDA.Mem.DeviceBuffer}}}}, Random._GLOBAL_RNG, Val{nothing}}((data = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; … ;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], label = Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0]), 1000, false, true, true, false, Val{nothing}(), Random._GLOBAL_RNG()), [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0

┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007f870fea9e40.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArraysCore ~/.julia/packages/GPUArraysCore/B3xv7/src/GPUArraysCore.jl:103


### コード13-17. `Flux.jl` 使用例(5)：MNISTデータの内容確認セットのロードと `DataLoader` の使用例

In [12]:
nt = first(traindata) |> cpu;

In [13]:
using ImageCore

[ Info: Precompiling ImageCore [a09fc81d-aa75-5fe9-8630-4744c3626534]


In [14]:
for i=1:3
    println("label: ", onecold(nt.label[:, i], 0:9))
    display(MNIST.convert2image(nt.data[:, :, 1, i]))
end

label: 6


label: 7


label: 4


### コード13-18. `Flux.jl` 使用例(6)：ロードしたデータと `model()`/`loss()`/`accuracy()` の動作確認

In [15]:
size(first(traindata).data)

(28, 28, 1, 1000)

In [16]:
model(first(traindata).data[:, :, :, 1:10])

10×10 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.103036   -0.124836    -0.0442622  …  -0.109493   -0.0849299   -0.115282
  0.0389069   0.0353087    0.0110856      0.0333007  -0.00123392   0.0505665
 -0.0404502  -0.00296133  -0.0600327     -0.111657    0.0100076   -0.078781
  0.0942768   0.0807195    0.0693815      0.0552367   0.0406068    0.0793996
 -0.0314369  -0.0574379    0.0039151     -0.0313798  -0.00454419  -0.0181142
  0.0476118   0.025042     0.0093353  …   0.0394195   0.0524659    0.0560288
  0.0616456   0.0326318    0.10864        0.0418966   0.0431191    0.087888
  0.0202037  -0.0300777    0.0406921      0.0532156  -0.0584709    0.0317761
 -0.0374669  -0.0166586   -0.0544326     -0.0474802   0.00298177  -0.0143879
 -0.171095   -0.212299    -0.180723      -0.139568   -0.185635    -0.201224

In [17]:
model(tX)

10×10000 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.134402    -0.0535021    -0.0250248   …  -0.154912    -0.144354
 -0.00318404   0.00771412   -0.00237338      0.0128761    0.0322657
 -0.00170011  -0.0896526    -0.0269967      -0.00977556  -0.0116152
  0.06401      0.0391987     0.0609606       0.0557247    0.0466434
 -0.0943012   -0.000788061  -0.0146994      -0.0541751   -0.0692916
 -0.00743502   0.0248189     0.0399536   …   0.0126711   -0.00431363
  0.0197424    0.0449761     0.0705727       0.0293096    0.107265
 -0.00658444   0.0074352     0.0443314       0.0200774   -0.038625
  8.64975f-5  -0.0543419    -0.0228404      -0.00066925   0.0815334
 -0.172284    -0.162676     -0.0890573      -0.123593    -0.189501

In [18]:
Loss(model)(tX, tY)

2.3113925f0

In [19]:
accuracy(onecold(model(tX)), onecold(tY))

0.1279

In [20]:
@info("metrics",accuracy=accuracy(onecold(model(tX)), onecold(tY)))

┌ Info: metrics
└   accuracy = 0.1279


### コード13-19. `Flux.jl` 使用例(7)：`train!()` 関数の整備とコールバックの準備

In [21]:
function train!(m, traindata; epochs = 10, cb = identity)
    loss = Loss(m)
    _params = Flux.params(m)
    opt = ADAM()
    @epochs epochs Flux.train!(loss, _params, traindata, opt; cb=cb)
end

train! (generic function with 1 method)

In [22]:
evalcb = throttle(() -> @info("metrics", accuracy=accuracy(onecold(model(tX)), onecold(tY))), 10)

(::Flux.var"#throttled#122"{Flux.var"#throttled#118#123"{Bool, Bool, var"#4#5", Int64}}) (generic function with 1 method)

### コード13-20. `Flux.jl` 使用例(8)：`train!()` の実行例

In [23]:
# train!(model, traindata; epochs=100, cb=evalcb)
# GPUが使えるなら↑こちらでもOK
train!(model, traindata; epochs=10, cb=evalcb)

[ Info: Epoch 1
┌ Info: metrics
└   accuracy = 0.3134
[ Info: Epoch 2
[ Info: Epoch 3
[ Info: Epoch 4
[ Info: Epoch 5
┌ Info: metrics
└   accuracy = 0.9891
[ Info: Epoch 6
[ Info: Epoch 7
[ Info: Epoch 8
[ Info: Epoch 9
┌ Info: metrics
└   accuracy = 0.9923
[ Info: Epoch 10


### コード13-21. `Flux.jl` 使用例(9)：学習結果の確認・推測

In [24]:
Loss(model)(tX, tY)

0.022840226f0

In [25]:
ŷ = model(tX) |> cpu |> onecold;

In [26]:
y = tY |> cpu |> onecold;

In [27]:
accuracy(ŷ, y)

0.9926

In [28]:
new_data = rand(Float32, 784);  # 実際には何らかの 28×28 のデータ

In [29]:
reshape(new_data, 28, 28, 1, 1) |> gpu |> model |> cpu |> (y->onecold(y, 0:9)) |> only

8

### コード13-22. `Flux.jl` 使用例(10)：混同行列・マクロ平均F値

In [30]:
using MLJBase: coerce, confusion_matrix, OrderedFactor

In [31]:
confusion_matrix(ŷ, coerce(y, OrderedFactor))  # 混同行列

10×10 Matrix{Int64}:
 976     0     2     0    0    2    3     0    2     0
   0  1132     1     0    0    0    2     3    0     2
   0     0  1024     0    1    0    0     1    0     0
   0     0     0  1006    0    3    0     0    0     0
   0     0     0     0  969    0    1     0    0     1
   0     0     0     3    0  881    1     0    0     3
   1     1     0     0    1    2  949     0    0     1
   1     2     2     0    0    1    0  1019    1     1
   2     0     3     1    1    2    2     1  970     1
   0     0     0     0   10    1    0     4    1  1000

In [32]:
# ↓前節のものと同じもの（再掲）
function fscoreM(ŷ, y)
    cfm = confusion_matrix(ŷ, coerce(y, OrderedFactor))
    axis = axes(cfm.mat, 1)
    tmp = [
        (;
            tp=cfm.mat[i, i],
            fp=sum(cfm.mat[i, axis.!=i]),
            fn=sum(cfm.mat[axis.!=i, i]),
            # tn=sum(cfm.mat[axis.!=i, axis.!=i])
        )
        for i in axis
    ]
    preM = mean(tp==fp==0 ? 0.0 : tp/float(tp+fp) for (;tp, fp) in tmp)
    recM = mean(tp==fn==0 ? 0.0 : tp/float(tp+fn) for (;tp, fn) in tmp)
    preM == recM == 0.0 ? 0.0 : 2 * preM * recM / (preM + recM)
end

fscoreM (generic function with 1 method)

In [33]:
fscoreM(ŷ, y)

0.9925375854099623